In [1]:
import torch

# 현재 사용 중인 GPU의 속성 정보 가져오기
gpu_stats = torch.cuda.get_device_properties(0)  # 첫 번째 GPU 속성 정보를 가져옴

# 현재 GPU에서 예약된 메모리 양을 GB 단위로 계산하여 출력
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"{start_gpu_memory} GB of memory reserved.")  # 예약된 메모리 양 출력

# GPU의 전체 메모리 크기를 GB 단위로 계산하여 출력
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")  # GPU 이름과 최대 메모리 출력

0.0 GB of memory reserved.
GPU = NVIDIA GeForce RTX 3060 Laptop GPU. Max memory = 6.0 GB.


In [2]:
from unsloth import FastLanguageModel
import torch
from peft import PeftModel

# 모델의 데이터 타입을 설정
dtype = torch.bfloat16

# 모델을 4비트 양자화하여 로드할지 여부
load_in_4bit = True

# 배치 크기 설정
BATCH_SIZE = 8

# 모델과 토크나이저 초기화
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Bllossom/llama-3.2-Korean-Bllossom-3B",
    max_seq_length=200,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    device_map="auto",
)

tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.46.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3060 Laptop GPU. Max memory: 6.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.6. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Bllossom/llama-3.2-Korean-Bllossom-3B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [3]:
from datasets import load_dataset

dataset = load_dataset("allenai/math_qa", trust_remote_code=True)

In [4]:
import re

def preprocess_options(dataset):
    
    processed_options = []
        
    for options_string in dataset["options"]:
        options = [
            re.sub(r'^[a-e] \) ', '', option.strip()).upper()
            for option in re.findall(r'[a-e] \) [^,]+', options_string)
        ]
        processed_options.append(options)
        
    return {
        "options" : processed_options
    }

In [5]:
train_data = dataset['train']
eval_data = dataset['validation']

In [6]:
train_dataset = train_data.map(preprocess_options, batched=True, num_proc=4,)
eval_dataset = eval_data.map(preprocess_options, batched=True, num_proc=4,)

In [7]:
train_dataset[0]

{'Problem': "the banker ' s gain of a certain sum due 3 years hence at 10 % per annum is rs . 36 . what is the present worth ?",
 'Rationale': '"explanation : t = 3 years r = 10 % td = ( bg × 100 ) / tr = ( 36 × 100 ) / ( 3 × 10 ) = 12 × 10 = rs . 120 td = ( pw × tr ) / 100 ⇒ 120 = ( pw × 3 × 10 ) / 100 ⇒ 1200 = pw × 3 pw = 1200 / 3 = rs . 400 answer : option a"',
 'options': ['RS . 400', 'RS . 300', 'RS . 500', 'RS . 350', 'NONE OF THESE'],
 'correct': 'a',
 'annotated_formula': 'divide(multiply(const_100, divide(multiply(36, const_100), multiply(3, 10))), multiply(3, 10))',
 'linear_formula': 'multiply(n2,const_100)|multiply(n0,n1)|divide(#0,#1)|multiply(#2,const_100)|divide(#3,#1)|',
 'category': 'gain'}

In [8]:
prompt_template = """
문제: {question}
A. {A}
B. {B}
C. {C}
D. {D}
정답:{answer}"""

EOS_TOKEN = tokenizer.eos_token

In [9]:
def preprocess_function(dataset):
    question = dataset['Problem']
    options_list = dataset['options']
    answers = dataset['correct']

    result = []
    
    for q, option, ans in zip(question, options_list, answers):
        A = option[0]
        B = option[1]
        C = option[2]
        D = option[3]
        
        ans = ans.upper()
        
        text = prompt_template.format(question=q, A=A, B=B, C=C, D=D, answer=ans)
        text += EOS_TOKEN
        result.append(text)
        
    return {
        "text" : result
    }

In [10]:
# dataset 에 preprocess_function 적용
train_dataset = train_dataset.map(preprocess_function, batched=True, num_proc=4, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(preprocess_function, batched=True, num_proc=4, remove_columns=eval_dataset.column_names)

Map (num_proc=4):   0%|          | 0/29837 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4475 [00:00<?, ? examples/s]

In [13]:
print(train_dataset['text'][0])


문제: the banker ' s gain of a certain sum due 3 years hence at 10 % per annum is rs . 36 . what is the present worth ?
A. RS . 400
B. RS . 300
C. RS . 500
D. RS . 350
정답:A<|eot_id|>


In [ ]:
# 모델 초기화
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # 0보다 큰 어떤 숫자도 선택 가능! 8, 16, 32, 64, 128이 권장됩니다.
    lora_alpha=16,  # LoRA 알파 값을 설정합니다. # 튜토리얼은 16
    lora_dropout=0.01,  # 드롭아웃을 지원합니다. # # Supports any, but = 0 is optimized
    target_modules=[
        "q_proj", # query
        "k_proj", # key
        "v_proj", # value
        "o_proj", # output (어텐션 최종 출력)
        "gate_proj", # 게이트 조절 시 사용
        "up_proj", # 차원 확장 시 사용
        "down_proj", # 차원 축소 시 사용
    ],  # 타겟 모듈을 지정합니다.
    bias="none",  # 바이어스를 지원합니다.
    # True 또는 "unsloth"를 사용하여 매우 긴 컨텍스트에 대해 VRAM을 30% 덜 사용하고, 2배 더 큰 배치 크기를 지원합니다.
    use_gradient_checkpointing="unsloth",
    random_state=3407,  # 난수 상태를 설정합니다. # 공식 : 3407
    use_rslora=True,  # 순위 안정화 LoRA를 지원합니다.
    loftq_config=None,  # LoftQ를 지원합니다.
)

In [ ]:
# model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# 총 스텝 수 계산
total_steps = len(train_dataset) // (4 * 4)  # batch_size * gradient_accumulation_steps

training_args = TrainingArguments(
    # 배치 설정
    per_device_train_batch_size=4,       # 훈련 시 배치 사이즈
    per_device_eval_batch_size=4,        # 평가 시 배치 사이즈
    gradient_accumulation_steps=4,       # gradient 누적 단계 수 (메모리 최적화)

    # 웜업 및 에폭
    warmup_ratio=0.1,                    # 전체 스텝 대비 warmup 비율
    num_train_epochs=1,                  # 총 학습 에폭 수

    # 최적화 및 학습률 스케줄링
    learning_rate=2e-4,                  # 초기 학습률
    optim="adamw_bnb_8bit",              # 8비트 AdamW 최적화
    weight_decay=0.01,                   # 가중치 감소율
    lr_scheduler_type="cosine",          # 코사인 학습률 스케줄러
    # lr_scheduler_type="cosine_with_restarts",  # 코사인 스케줄러에 재시작 추가 가능

    # 하드웨어 호환이 필요한 최적화
    fp16=not torch.cuda.is_bf16_supported(),  # FP16 사용 (BF16 미지원 시)
    bf16=torch.cuda.is_bf16_supported(),      # BF16 사용 (지원 시)
    torch_compile=True,                       # PyTorch 컴파일을 통한 메모리, 학습 최적화

    # 로깅 및 wandb
    logging_steps=total_steps // 20,       # 전체 스텝의 5%마다 로깅
    report_to="wandb",                    # wandb
    run_name="KMMLU_ALL_CATEGORIES",      # run 이름

    # 평가 및 체크포인트
    eval_strategy="steps",                # 정기적인 평가
    eval_steps=total_steps // 20,         # 전체 스텝의 5%마다 평가
    load_best_model_at_end=True,          # 최고 성능 모델 저장
    metric_for_best_model="eval_loss",    # 성능 평가 기준 (loss)
    greater_is_better=False,              # 낮은 loss가 더 좋은 모델로 평가

    # 체크포인트 저장
    save_strategy="steps",                # 체크포인트 저장 전략 (스텝 단위)
    save_steps=total_steps // 20,         # 전체 스텝의 5%마다 저장
    save_total_limit=5,                   # 최대 체크포인트 저장 개수

    # 기타 설정
    seed=3407,                            # 시드 고정
    output_dir="outputs",                 # 출력 경로
    max_grad_norm=1.0,                    # 최대 gradient norm 설정
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    dataset_num_proc=4,
    packing=False,
    args=training_args,
)

In [ ]:
import wandb

wandb.login()

trainer_stats = trainer.train()

wandb.finish()